# Define tool and model of the tool

In [1]:
!nvidia-smi

Mon Sep 25 15:14:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   57C    P0   245W / 300W |  32501MiB / 32510MiB |     97%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   

In [2]:
import os, torch
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'
os.environ['CUDA_VISIBLE_DEVICES'] = '6'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
import sys

NAME = "idk-mrc"
NO_ANSWER_STATEMENT = "Tidak ada jawaban"

TASK_NER_NAME = "ner"
MODEL_NER_NAME = "ageng-anugrah/indobert-large-p2-finetuned-ner"

TASK_CHUNKING_NAME = "token-classification"
MODEL_CHUNKING_NAME = "ageng-anugrah/indobert-large-p2-finetuned-chunking"

MODEL_SIMILARITY_NAME = "paraphrase-multilingual-mpnet-base-v2"
URL_STOPWORD = "https://raw.githubusercontent.com/6/stopwords-json/master/stopwords-all.json"

TASK_PARAPHRASER_NAME = "text2text-generation"
MODEL_PARAPHRASER_NAME = ""

# SAMPLE = sys.maxsize
SAMPLE = 100

# Import anything

In [4]:
import transformers
import evaluate
import torch
import operator
import re
import sys
import collections
import string
import contextlib
import gc
import random
import string
import requests

import numpy as np
import pandas as pd
import torch.nn as nn

from multiprocessing import cpu_count
from evaluate import load
from nusacrowd import NusantaraConfigHelper
from datetime import datetime
from huggingface_hub import notebook_login
from tqdm import tqdm
from huggingface_hub import HfApi
from sentence_transformers import SentenceTransformer, util

from datasets import (
    load_dataset, 
    Dataset,
    DatasetDict
)
from transformers import (
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EarlyStoppingCallback, 
    AutoModelForQuestionAnswering,
    AutoModelForTokenClassification,
    pipeline
)

# Retrieve QA dataset

In [5]:
print("PROGRAM STARTED")

PROGRAM STARTED


In [6]:
conhelps = NusantaraConfigHelper()
data_qas = conhelps.filtered(lambda x: 'idk_mrc' in x.dataset_name)[0].load_dataset()

df_train = pd.DataFrame(data_qas['train'])
df_validation = pd.DataFrame(data_qas['validation'])
df_test = pd.DataFrame(data_qas['test'])

cols = ['context', 'question', 'answer']
new_df_train = pd.DataFrame(columns=cols)

for i in tqdm(range(len(df_train['context']))):
    for j in df_train["qas"][i]:
        if len(j['answers']) != 0:
            new_df_train = new_df_train.append({'context': df_train["context"][i], 
                                                'question': j['question'], 
                                                'answer': {"text": j['answers'][0]['text'], 
                                                           "answer_start": j['answers'][0]['answer_start'], 
                                                           "answer_end": j['answers'][0]['answer_start'] + len(j['answers'][0]['text'])}}, 
                                                           ignore_index=True)
        else:
            new_df_train = new_df_train.append({'context': df_train["context"][i], 
                                                'question': j['question'], 
                                                'answer': {"text": str(), 
                                                           "answer_start": 0, 
                                                           "answer_end": 0}}, 
                                                           ignore_index=True)

cols = ['context', 'question', 'answer']
new_df_val = pd.DataFrame(columns=cols)

for i in tqdm(range(len(df_validation['context']))):
    for j in df_validation["qas"][i]:
        if len(j['answers']) != 0:
            new_df_val = new_df_val.append({'context': df_validation["context"][i], 
                                            'question': j['question'], 
                                            'answer': {"text": j['answers'][0]['text'], 
                                                       "answer_start": j['answers'][0]['answer_start'], 
                                                       "answer_end": j['answers'][0]['answer_start'] + len(j['answers'][0]['text'])}}, 
                                                       ignore_index=True)
        else:
            new_df_val = new_df_val.append({'context': df_validation["context"][i], 
                                            'question': j['question'], 
                                            'answer': {"text": str(), 
                                                       "answer_start": 0, 
                                                       "answer_end": 0}}, 
                                                       ignore_index=True)        

cols = ['context', 'question', 'answer']
new_df_test = pd.DataFrame(columns=cols)

for i in tqdm(range(len(df_test['context']))):
    for j in df_test["qas"][i]:
        if len(j['answers']) != 0:
            new_df_test = new_df_test.append({'context': df_test["context"][i], 
                                            'question': j['question'], 
                                            'answer': {"text": j['answers'][0]['text'], 
                                                       "answer_start": j['answers'][0]['answer_start'], 
                                                       "answer_end": j['answers'][0]['answer_start'] + len(j['answers'][0]['text'])}}, 
                                                       ignore_index=True)
        else:
            new_df_test = new_df_test.append({'context': df_test["context"][i], 
                                            'question': j['question'], 
                                            'answer': {"text": str(), 
                                                       "answer_start": 0, 
                                                       "answer_end": 0}}, 
                                                       ignore_index=True)

train_dataset = Dataset.from_dict(new_df_train)
validation_dataset = Dataset.from_dict(new_df_val)
test_dataset = Dataset.from_dict(new_df_test)

data_qas = DatasetDict({"train": train_dataset, "validation": validation_dataset, "test": test_dataset})
data_qas

  0%|          | 0/3 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 378/378 [00:01<00:00, 269.06it/s]


DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 9332
    })
    validation: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 764
    })
    test: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 844
    })
})

# Convert to NLI, with hypothesis being just do concat question & answer

## Convert Dataset to DataFrame format

In [7]:
seed_value = 42
random.seed(seed_value)

In [8]:
if SAMPLE == sys.maxsize:
    data_qas_train_df = pd.DataFrame(data_qas["train"][:SAMPLE])
    data_qas_val_df = pd.DataFrame(data_qas["validation"][:SAMPLE])
    data_qas_test_df = pd.DataFrame(data_qas["test"][:SAMPLE])

else:
    data_qas_train_df = (pd.DataFrame(data_qas["train"])).sample(n=SAMPLE, random_state=42)
    data_qas_val_df = (pd.DataFrame(data_qas["validation"])).sample(n=SAMPLE, random_state=42)
    data_qas_test_df = (pd.DataFrame(data_qas["test"])).sample(n=SAMPLE, random_state=42)

    data_qas_train_df = data_qas_train_df.reset_index(drop=True)
    data_qas_val_df = data_qas_val_df.reset_index(drop=True)
    data_qas_test_df = data_qas_test_df.reset_index(drop=True)

## Retrieve answer text only

In [9]:
def retrieve_answer_text(data):
    for i in range(len(data)):
        data['answer'][i] = data['answer'][i]['text']
    return data

In [10]:
data_qas_train_df = retrieve_answer_text(data_qas_train_df)
data_qas_val_df = retrieve_answer_text(data_qas_val_df)
data_qas_test_df = retrieve_answer_text(data_qas_test_df)

## Create NLI dataset from copy of QA dataset above

In [11]:
data_nli_train_df = data_qas_train_df.copy()
data_nli_val_df = data_qas_val_df.copy()
data_nli_test_df = data_qas_test_df.copy()

## Convert context pair to premise (only renaming column)

In [12]:
data_nli_train_df = data_nli_train_df.rename(columns={"context": "premise"})
data_nli_val_df = data_nli_val_df.rename(columns={"context": "premise"})
data_nli_test_df = data_nli_test_df.rename(columns={"context": "premise"})

# Add contradiction label cases

## Import pipeline to create contradiction cases

In [13]:
nlp_tools_ner = pipeline(task = TASK_NER_NAME, 
                     model = MODEL_NER_NAME, 
                     tokenizer = AutoTokenizer.from_pretrained(MODEL_NER_NAME, 
                                                               model_max_length=512, 
                                                               truncation=True),
                     aggregation_strategy = 'simple')

In [14]:
nlp_tools_chunking = pipeline(task = TASK_CHUNKING_NAME, 
                     model = MODEL_CHUNKING_NAME, 
                     tokenizer = AutoTokenizer.from_pretrained(MODEL_CHUNKING_NAME, 
                                                               model_max_length=512, 
                                                               truncation=True),
                     aggregation_strategy = 'simple')

## Add NER and chunking tag column in DataFrame

In [15]:
def remove_space_after_number_and_punctuation(text):
    pattern = r'(\d+)\s*([.,])\s*(?=\S|$)'
    cleaned_text = re.sub(pattern, r'\1\2', text)
    return cleaned_text

In [16]:
def add_premise_tag(data, tag, index, premise_array, ner=nlp_tools_ner, chunking=nlp_tools_chunking):

    if tag == "ner": tools=ner
    else: tools=chunking
    
    if len(tools(data['premise'][index])) == 0:
        premise_array.append("NO TOKEN DETECTED")
    
    else:
        for j in tools(data['premise'][index]):
            tag_premise = (j['entity_group'], remove_space_after_number_and_punctuation(j['word']))
            premise_array.append(tag_premise)

    return premise_array

In [17]:
def remove_punctuation(text):
    return text.strip(string.punctuation)

In [18]:
def add_row_tag(answer, tag, premise_array, ner=nlp_tools_ner, chunking=nlp_tools_chunking):

    if tag == "ner": tools=ner
    else: tools=chunking

    tag_answer_list = []
    
    if tag == "ner":
        if len(premise_array) != 0:
            for i in premise_array:

                label_from_premise_tag = i[0]
                word_from_premise_tag = remove_space_after_number_and_punctuation(i[1])

                splitted_word_from_premise_tag = set(remove_punctuation(text) for text in word_from_premise_tag.split())

                # With assumption, that I do not divide label when
                # there is more than one label in one word answer.
                # Instead, I give a NULL.

                if word_from_premise_tag.lower() == answer.lower():
                    tag_answer = (label_from_premise_tag, word_from_premise_tag)
                    break

                # Or, I could do this: to reducing NULL label with char not really with string.
                # But, the tradeoff is real-answer can be replace with word in premise_array
                elif answer.lower() in word_from_premise_tag:
                    tag_answer = (label_from_premise_tag, answer.lower())
                    break

                else:
                    tag_answer = ("NULL", answer)
            tag_answer_list.append(tag_answer)

        else:
            tag_answer = ("NULL", answer)
            tag_answer_list.append(tag_answer)
    
    elif tag == "chunking":
        retrieved_from_tools = tools(answer)

        if len(retrieved_from_tools) != 0:
            for i in retrieved_from_tools:
                tag_answer = (i['entity_group'], i['word'])
                tag_answer_list.append(tag_answer)
        else:
            tag_answer = ("NULL", answer)
            tag_answer_list.append(tag_answer)
            
        #sequence = [value[0] for value in tag_answer_list]
        #tag_answer_list.append(sequence)
        
    return tag_answer_list

In [19]:
def find_order(list1, list2):
    i = 0
    j = 0
    matching_values = []

    while i < len(list1) and j < len(list2):
        if list1[i][0] == list2[j][0]:
            matching_values.append(list2[j][1])
            j += 1
        i += 1

    if j == len(list2):
        return " ".join(matching_values)
    else:
        return None

In [20]:
arr_premise = [('PLACE', 'kota ho chi minh'), ('PLACE', 'vietnam'), ('PLACE', 'thanh pho ho chi minh'), ('PLACE', 'vietnam'), ('PLACE', 'sungai mekong'), ('PLACE', 'prey nok'), ('PLACE', 'kamboja'), ('PLACE', 'vietnam'), ('PLACE', 'saigon'), ('PLACE', 'vietnam'), ('PLACE', 'koloni perancis cochinchina'), ('PLACE', 'vietnam selatan'), ('PLACE', 'saigon'), ('PLACE', 'provinsi gia'), ('PLACE', 'kota ho chi minh'), ('PLACE', 'saigon'), ('PLACE', 'sungai saigon'), ('PLACE', 'china selatan')]

x = add_row_tag("Ho Chi Minh", "ner", arr_premise)
x

[('PLACE', 'ho chi minh')]

In [21]:
add_row_tag(
    answer="ruang dalam sel yang berisi cairan (cell sap dalam bahasa Inggris) yang berupa rongga yang diselaputi membran", 
    tag="chunking", 
    premise_array=[('NP', 'vakuola'), ('VP', 'merupakan'), ('NP', 'ruang'), ('PP', 'dalam'), ('NP', 'sel'), ('SBAR', 'yang'), ('VP', 'berisi'), ('NP', 'cairan ( cell sap'), ('PP', 'dalam'), ('NP', 'bahasa inggris )'), ('SBAR', 'yang'), ('VP', 'berupa'), ('NP', 'rongga'), ('SBAR', 'yang'), ('VP', 'diselaputi'), ('NP', 'membran ( tonoplas )'), ('NP', 'cairan ini'), ('VP', 'adalah'), ('NP', 'air'), ('PP', 'di'), ('NP', 'dalamnya'), ('VP', 'terlarut'), ('NP', 'zat'), ('PP', 'seperti'), ('NP', 'enzim, lipid, alkaloid, garam mineral, asam, dan basa'), ('PP', 'selain'), ('NP', 'itu'), ('NP', 'vakuola'), ('ADVP', 'juga'), ('VP', 'berisi'), ('NP', 'asam organik, asam amino, glukosa, dan gas'), ('NP', 'vakuola'), ('VP', 'ditemukan'), ('PP', 'pada'), ('NP', 'semua sel tumbuhan'), ('ADVP', 'tidak'), ('VP', 'dijumpai'), ('PP', 'pada'), ('NP', 'sel hewan dan bakteri'), ('VP', ','), ('ADVP', 'kecuali'), ('PP', 'pada'), ('NP', 'hewan uniseluler tingkat rendah')])

[('NP', 'ruang'),
 ('PP', 'dalam'),
 ('NP', 'sel'),
 ('SBAR', 'yang'),
 ('VP', 'berisi'),
 ('NP', 'cairan ( cell sap'),
 ('PP', 'dalam'),
 ('NP', 'bahasa inggris )'),
 ('SBAR', 'yang'),
 ('VP', 'berupa'),
 ('NP', 'rongga'),
 ('SBAR', 'yang'),
 ('VP', 'diselaputi'),
 ('NP', 'membran')]

In [22]:
def add_ner_and_chunking_all_tag(data):
    
    data['ner_tag_answer'] = ""
    data['chunking_tag_answer'] = ""
    
    data['ner_tag_premise'] = ""
    data['chunking_tag_premise'] = ""
    
    for i in tqdm(range(len(data))):
        
        answer = data['answer'][i]
        premise = data['premise'][i]
        
        ner_premise_array = []
        chunking_premise_array = []
                                                
        data['ner_tag_premise'][i] = add_premise_tag(data, "ner", i, ner_premise_array)
        data['chunking_tag_premise'][i] = add_premise_tag(data, "chunking", i, chunking_premise_array)
        
        data['ner_tag_answer'][i] = add_row_tag(answer, "ner", data['ner_tag_premise'][i])
        data['chunking_tag_answer'][i] = add_row_tag(answer, "chunking", data['chunking_tag_premise'][i])
    
    return data

In [ ]:
data_nli_train_df = add_ner_and_chunking_all_tag(data_nli_train_df)
data_nli_val_df = add_ner_and_chunking_all_tag(data_nli_val_df)
data_nli_test_df = add_ner_and_chunking_all_tag(data_nli_test_df)

 31%|█████████████████████████▍                                                        | 31/100 [01:51<04:08,  3.60s/it]

# Create wrong answer for answerable

This is the flow to create wrong answer:

1. Check the NER and POS/Chunking labels of the right_answer and context/premise.

2. Search and group NER and POS/Chunking labels that match the right_answer throughout the context/premise.

3. Perform NER classification. There will be two branches here, namely:

   3a. If the NER of the right_answer can be detected, then calculate the distance using semantic similarity or word vectors between the right_answer and various possible wrong_answers with the same NER as the right_answer. Once done, proceed to the final wrong_answer.
   
   3b. If the NER of the right_answer cannot be detected (NULL) or context/premise does not contain any of NER of right_answer, then the POS/Chunking of the right_answer will be identified.
   
4. Perform POS/Chunking classification. Continuation from point 3b. There will be two more branches:

   4a. If the POS/Chunking of the right_answer can be detected, then calculate the distance using semantic similarity or word vectors between the right_answer and various possible wrong_answers with the same POS/Chunking as the right_answer. Once done, proceed to the final wrong_answer.
   
   4b. If the POS/Chunking of the right_answer cannot be detected (NULL) or context/premise does not contain any of NER of right_answer, then the final wrong_answer will be chosen based on a random word (random_word) from the context/premise.

In [ ]:
model_similarity = SentenceTransformer(MODEL_SIMILARITY_NAME)

def return_similarity_sorted_array(right_answer, sentence_array, model=model_similarity):
    
    right_answer = right_answer.lower()
    
    embedding_right_answer = model.encode([right_answer], convert_to_tensor=True, device=device)
    embedding_sentence_array = model.encode(sentence_array, convert_to_tensor=True, device=device)
    
    cosine_scores = util.pytorch_cos_sim(embedding_right_answer, embedding_sentence_array)
    
    sorted_indices = cosine_scores.argsort(descending=True)[0]
    sorted_array = [sentence_array[i] for i in sorted_indices]
    
    return sorted_array

In [ ]:
def remove_values_with_hash(arr):
    return [item for item in arr if "#" not in item]

In [ ]:
response = requests.get(URL_STOPWORD)

if response.status_code == 200:
    stopword_data = response.json()
else:
    print("Failed to download JSON.")

stopword_data = set([item for sublist in list(stopword_data.values()) for item in sublist])

In [ ]:
def select_random_word(text, stopword_data=stopword_data):
    words = re.findall(r'\w+', text.lower())
    filtered_words = [word for word in words if word not in stopword_data and word not in string.punctuation]
    random_word = random.choice(filtered_words)
    return random_word

In [ ]:
def grouping_same_tag(tag_answers, tag_premises, same_tag_array, tag):
    
    if tag == "ner":
        for tag_premise in tag_premises:

            label_tag_premise = tag_premise[0]
            word_premise = tag_premise[1]

            for tag_answer in tag_answers:

                label_tag_answer = tag_answer[0]

                if label_tag_answer == label_tag_premise:
                    same_tag_array.append(word_premise)
    
    elif tag == "chunking":
        matching_words = find_order(tag_premises, tag_answers)
        if matching_words != None:
            same_tag_array.append(matching_words)
        else:
            grouping_same_tag(tag_answers, tag_premises, same_tag_array, "ner")

    return remove_values_with_hash(same_tag_array)

In [ ]:
def remove_punctuation(text):
    return text.strip(string.punctuation)

In [ ]:
def contains_only_punctuation(text):
    return all(char in string.punctuation for char in text)

In [ ]:
def filtering_plausible_answer(answer, plausible_answer_array):
    
    answer = answer.lower()
    
    plausible_answer_array = [item.lower().strip() for item in plausible_answer_array]
    plausible_answer_array = [string for string in plausible_answer_array if not contains_only_punctuation(string)]
    plausible_answer_array = [remove_punctuation(text) for text in plausible_answer_array]
    
    final_plausible_answer_array = []
    answer_words = set(remove_punctuation(text) for text in answer.split())
    
    for plausible_answer in plausible_answer_array:
        plausible_answer_words = set(plausible_answer.split())
        if not plausible_answer_words.intersection(answer_words):
            final_plausible_answer_array.append(plausible_answer)
    
    return final_plausible_answer_array

In [ ]:
def is_number(input_str):
    pattern = r'^-?\d+(\.\d+)?$'
    return re.match(pattern, input_str) is not None

def is_date(input_str):
    pattern = r'^\d{4}-\d{2}-\d{2}$'
    return re.match(pattern, input_str) is not None

def is_time(input_str):
    pattern = r'^\d{2}:\d{2}:\d{2}$'
    return re.match(pattern, input_str) is not None

def check_regex(right_answer, plausible_answer_array):
    
    if is_number(right_answer):
        plausible_answer_array = [item for item in plausible_answer_array if is_number(item)]
    
    elif is_date(right_answer):
        plausible_answer_array = [item for item in plausible_answer_array if is_date(item)]
    
    elif is_time(right_answer):
        plausible_answer_array = [item for item in plausible_answer_array if is_time(item)]
    
    return plausible_answer_array

In [ ]:
def sorting_similarity(data, right_answer, index, tag, plausible_answer_array, premise):

    if tag == "ner": slice='same_ner_tag_answer'
    elif tag == "chunking": slice='same_chunking_tag_answer'
    else: slice=None

    # Find all the sorted (by similarity) plausible wrong answer, 
    # and remove hask & punctuation only answer
    if slice != None:
        wrong_answer_array = return_similarity_sorted_array(right_answer, data[slice][index])
    else:
        wrong_answer_array = return_similarity_sorted_array(right_answer, plausible_answer_array)
    
    plausible_answer_array = remove_values_with_hash(wrong_answer_array)
    plausible_answer_array = filtering_plausible_answer(right_answer, plausible_answer_array)
    plausible_answer_array = check_regex(right_answer, plausible_answer_array)
    
    try:
        # Only return the most similar to right_answer
        wrong_answer = plausible_answer_array[0].strip()
        
        if tag == "ner": 
            properties = "IDENTICAL NER labels were found, and the highest similarity score same NER array was selected"
        elif tag == "chunking":
            properties = "IDENTICAL Chunking labels were found, and the highest similarity score from same Chunking array was selected"
        else:
            properties = "NO CHUNKING labels were found, and the highest similarity score from plausible answer was selected"
    except:
        wrong_answer = select_random_word(premise)
        
        if tag == "ner": 
            properties = "Detected (NER) wrong answer that is the SAME as the right answer, search random word from premise"
        elif tag == "chunking":
            properties = "Detected (Chunking) wrong answer that is the SAME as the right answer, search random word from premise"
        else:
            properties = "Detected (Random) wrong answer that is the SAME as the right answer, search random word from premise"
    
    assert isinstance(wrong_answer, str)
    assert isinstance(plausible_answer_array, list)
    
    return wrong_answer, plausible_answer_array, properties

In [ ]:
def create_wrong_answer(data, NO_ANSWER_STATEMENT=NO_ANSWER_STATEMENT):
    
    data['same_ner_tag_answer'] = ""
    data['same_chunking_tag_answer'] = ""
    data['wrong_answer'] = ""
    data['no_answer'] = ""
    data['plausible_answer_based_on_method'] = ""
    data['properties'] = ""
    
    for i in tqdm(range(len(data))):
        
        right_answer = data['answer'][i]
        premise = data['premise'][i]

        same_ner_tag_answer_array = []
        same_chunking_tag_answer_array = []

        ner_tag_answer = data['ner_tag_answer'][i]
        ner_tag_premise = data['ner_tag_premise'][i]

        chunking_tag_answer = data['chunking_tag_answer'][i]
        chunking_tag_premise = data['chunking_tag_premise'][i]
        
        if right_answer == "":
            data['properties'][i] = "Unanswerable question"
            data['wrong_answer'][i] = "NULL"
            data['no_answer'][i] = "NULL"
            data['plausible_answer_based_on_method'][i] = "Unanswerable question"
            continue
            
        # Grouped with the same NER & Chunking group, between answer and word of premise
        data['same_ner_tag_answer'][i] = grouping_same_tag(ner_tag_answer,
                                                           ner_tag_premise,
                                                           same_ner_tag_answer_array, "ner")
        
        data['same_chunking_tag_answer'][i] = grouping_same_tag(chunking_tag_answer, 
                                                                chunking_tag_premise, 
                                                                same_chunking_tag_answer_array, "chunking")
        
        # Start to create wrong answer
        plausible_answer_array = []

        # Perform NER classification
        # If the NER of the right_answer can be detected, then calculate the distance using semantic 
        # similarity or word vectors between the right_answer and various possible wrong_answers with 
        # the same NER as the right_answer. Once done, proceed to the final wrong_answer.
        if data['same_ner_tag_answer'][i] != []:
            wrong_answer, plausible_answer_array, properties = sorting_similarity(data, right_answer, \
                                                                      i, "ner", plausible_answer_array, premise)
            
        # If the NER of the right_answer cannot be detected (NULL) or context/premise does not contain 
        # any of NER of right_answer, then the POS/Chunking of the right_answer will be identified.
        # Perform POS/Chunking classification
        else:
            # If the POS/Chunking of the right_answer can be detected, then calculate the distance 
            # using semantic similarity or word vectors between the right_answer and various possible 
            # wrong_answers with the same POS/Chunking as the right_answer. Once done, proceed to the 
            # final wrong_answer.
            if data['same_chunking_tag_answer'][i] != []:
                wrong_answer, plausible_answer_array, properties = sorting_similarity(data, right_answer, \
                                                                          i, "chunking", plausible_answer_array, premise)
            
            # If the POS/Chunking of the right_answer cannot be detected (NULL) or context/premise 
            # does not contain any of NER of right_answer, then the final wrong_answer will be chosen 
            # based on a random word (random_word) from the context/premise.
            else:
                for chunking_tag in chunking_tag_premise:
                    plausible_answer_array.append(chunking_tag[1])

                wrong_answer, plausible_answer_array, properties = sorting_similarity(data, right_answer, \
                                                                          i, "none", plausible_answer_array, premise)
        data['properties'][i] = properties
        data['wrong_answer'][i] = wrong_answer
        data['no_answer'][i] = NO_ANSWER_STATEMENT
        data['plausible_answer_based_on_method'][i] = plausible_answer_array
            
    return data       

In [ ]:
data_nli_train_df = create_wrong_answer(data_nli_train_df)
data_nli_val_df = create_wrong_answer(data_nli_val_df)
data_nli_test_df = create_wrong_answer(data_nli_test_df)

# Split to two dataset: right dataset & wrong dataset

In [ ]:
def move_to_column_number(data, column_name="hypothesis", column_num=3):

    cols = list(data.columns)
    cols.remove(column_name)
    cols.insert(column_num, column_name)

    data = data[cols]
    
    return data

In [ ]:
columns_to_exclude = ['wrong_answer', 'no_answer']

data_nli_answerable_right_train_df = data_nli_train_df.drop(columns=columns_to_exclude).copy()
data_nli_answerable_right_val_df = data_nli_val_df.drop(columns=columns_to_exclude).copy()
data_nli_answerable_right_test_df = data_nli_test_df.drop(columns=columns_to_exclude).copy()

data_nli_answerable_right_train_df = data_nli_answerable_right_train_df[data_nli_answerable_right_train_df['answer'] != '']
data_nli_answerable_right_val_df = data_nli_answerable_right_val_df[data_nli_answerable_right_val_df['answer'] != '']
data_nli_answerable_right_test_df = data_nli_answerable_right_test_df[data_nli_answerable_right_test_df['answer'] != '']

data_nli_answerable_right_train_df = data_nli_answerable_right_train_df.reset_index(drop=True)
data_nli_answerable_right_val_df = data_nli_answerable_right_val_df.reset_index(drop=True)
data_nli_answerable_right_test_df = data_nli_answerable_right_test_df.reset_index(drop=True)

In [ ]:
columns_to_exclude = ['answer', 'no_answer']

data_nli_answerable_wrong_train_df = data_nli_train_df[data_nli_train_df['answer'] != '']
data_nli_answerable_wrong_val_df = data_nli_val_df[data_nli_val_df['answer'] != '']
data_nli_answerable_wrong_test_df = data_nli_test_df[data_nli_test_df['answer'] != '']

data_nli_answerable_wrong_train_df = data_nli_answerable_wrong_train_df.drop(columns=columns_to_exclude).copy()
data_nli_answerable_wrong_val_df = data_nli_answerable_wrong_val_df.drop(columns=columns_to_exclude).copy()
data_nli_answerable_wrong_test_df = data_nli_answerable_wrong_test_df.drop(columns=columns_to_exclude).copy()

data_nli_answerable_wrong_train_df.rename(columns={'wrong_answer': 'answer'}, inplace=True)
data_nli_answerable_wrong_val_df.rename(columns={'wrong_answer': 'answer'}, inplace=True)
data_nli_answerable_wrong_test_df.rename(columns={'wrong_answer': 'answer'}, inplace=True)

data_nli_answerable_wrong_train_df = data_nli_answerable_wrong_train_df.reset_index(drop=True)
data_nli_answerable_wrong_val_df = data_nli_answerable_wrong_val_df.reset_index(drop=True)
data_nli_answerable_wrong_test_df = data_nli_answerable_wrong_test_df.reset_index(drop=True)

data_nli_answerable_wrong_train_df = move_to_column_number(data_nli_answerable_wrong_train_df, "answer", 2)
data_nli_answerable_wrong_val_df = move_to_column_number(data_nli_answerable_wrong_val_df, "answer", 2)
data_nli_answerable_wrong_test_df = move_to_column_number(data_nli_answerable_wrong_test_df, "answer", 2)

In [ ]:
columns_to_exclude = ['wrong_answer', 'no_answer']

data_nli_unanswerable_right_train_df = data_nli_train_df.drop(columns=columns_to_exclude).copy()
data_nli_unanswerable_right_val_df = data_nli_val_df.drop(columns=columns_to_exclude).copy()
data_nli_unanswerable_right_test_df = data_nli_test_df.drop(columns=columns_to_exclude).copy()

data_nli_unanswerable_right_train_df = data_nli_unanswerable_right_train_df[data_nli_unanswerable_right_train_df['answer'] == '']
data_nli_unanswerable_right_val_df = data_nli_unanswerable_right_val_df[data_nli_unanswerable_right_val_df['answer'] == '']
data_nli_unanswerable_right_test_df = data_nli_unanswerable_right_test_df[data_nli_unanswerable_right_test_df['answer'] == '']

data_nli_unanswerable_right_train_df = data_nli_unanswerable_right_train_df.reset_index(drop=True)
data_nli_unanswerable_right_val_df = data_nli_unanswerable_right_val_df.reset_index(drop=True)
data_nli_unanswerable_right_test_df = data_nli_unanswerable_right_test_df.reset_index(drop=True)

In [ ]:
columns_to_exclude = ['answer', 'wrong_answer']

data_nli_unanswerable_wrong_train_df = data_nli_train_df[data_nli_train_df['answer'] != '']
data_nli_unanswerable_wrong_val_df = data_nli_val_df[data_nli_val_df['answer'] != '']
data_nli_unanswerable_wrong_test_df = data_nli_test_df[data_nli_test_df['answer'] != '']

data_nli_unanswerable_wrong_train_df = data_nli_unanswerable_wrong_train_df.drop(columns=columns_to_exclude).copy()
data_nli_unanswerable_wrong_val_df = data_nli_unanswerable_wrong_val_df.drop(columns=columns_to_exclude).copy()
data_nli_unanswerable_wrong_test_df = data_nli_unanswerable_wrong_test_df.drop(columns=columns_to_exclude).copy()

data_nli_unanswerable_wrong_train_df.rename(columns={'no_answer': 'answer'}, inplace=True)
data_nli_unanswerable_wrong_val_df.rename(columns={'no_answer': 'answer'}, inplace=True)
data_nli_unanswerable_wrong_test_df.rename(columns={'no_answer': 'answer'}, inplace=True)

data_nli_unanswerable_wrong_train_df = data_nli_unanswerable_wrong_train_df.sample(len(data_nli_unanswerable_right_train_df), replace=True)
data_nli_unanswerable_wrong_val_df = data_nli_unanswerable_wrong_val_df.sample(len(data_nli_unanswerable_right_val_df), replace=True)
data_nli_unanswerable_wrong_test_df = data_nli_unanswerable_wrong_test_df.sample(len(data_nli_unanswerable_right_test_df), replace=True)

data_nli_unanswerable_wrong_train_df = data_nli_unanswerable_wrong_train_df.reset_index(drop=True)
data_nli_unanswerable_wrong_val_df = data_nli_unanswerable_wrong_val_df.reset_index(drop=True)
data_nli_unanswerable_wrong_test_df = data_nli_unanswerable_wrong_test_df.reset_index(drop=True)

data_nli_unanswerable_wrong_train_df = move_to_column_number(data_nli_unanswerable_wrong_train_df, "answer", 2)
data_nli_unanswerable_wrong_val_df = move_to_column_number(data_nli_unanswerable_wrong_val_df, "answer", 2)
data_nli_unanswerable_wrong_test_df = move_to_column_number(data_nli_unanswerable_wrong_test_df, "answer", 2)

In [ ]:
print("ENTAILMENT ANSWERABLE")
print("TRAIN:", len(data_nli_answerable_right_train_df))
print("VAL:", len(data_nli_answerable_right_val_df))
print("TEST:", len(data_nli_answerable_right_test_df))
print()

print("CONTRADICTION ANSWERABLE")
print("TRAIN:", len(data_nli_answerable_wrong_train_df))
print("VAL:", len(data_nli_answerable_wrong_val_df))
print("TEST:", len(data_nli_answerable_wrong_test_df))
print()

print("ENTAILMENT UN-ANSWERABLE")
print("TRAIN:", len(data_nli_unanswerable_right_train_df))
print("VAL:", len(data_nli_unanswerable_right_val_df))
print("TEST:", len(data_nli_unanswerable_right_test_df))
print()

print("CONTRADICTION UN-ANSWERABLE")
print("TRAIN:", len(data_nli_unanswerable_wrong_train_df))
print("VAL:", len(data_nli_unanswerable_wrong_val_df))
print("TEST:", len(data_nli_unanswerable_wrong_test_df))

# Convert question-answer pair to hypothesis

In [ ]:
#nlp_tools_paraphraser = pipeline(task = TASK_PARAPHRASER_NAME, 
#                     model = MODEL_PARAPHRASER_NAME, 
#                     tokenizer = AutoTokenizer.from_pretrained(MODEL_PARAPHRASER_NAME, 
#                                                               model_max_length=512, 
#                                                               truncation=True))

In [ ]:
def convert_question_and_answer_to_hypothesis(data, NO_ANSWER_STATEMENT=NO_ANSWER_STATEMENT):
    
    data['hypothesis'] = ""
    hypothesis_array = []
    
    for i in range(len(data)):
        
        if data['answer'][i] != "" and type(data['answer'][i]) == str:
            hypothesis_array.append(data['question'][i] + ' ' + data['answer'][i])
        else:
            hypothesis_array.append(data['question'][i] + ' ' + NO_ANSWER_STATEMENT)
        
        # Use this to decline no-answer-warning properties
        #hypothesis_array.append(data['question'][i] + ' ' + data['answer'][i])
        
        # Use this to use paraphraser
        #hypothesis_array.append(str(nlp_tools_paraphraser(data['question'][i] + ' ' + data['answer'][i])[0]['generated_text']))
    
    data['hypothesis'] = hypothesis_array
    
    return data

In [ ]:
data_nli_answerable_right_train_df = convert_question_and_answer_to_hypothesis(data_nli_answerable_right_train_df)
data_nli_answerable_right_val_df = convert_question_and_answer_to_hypothesis(data_nli_answerable_right_val_df)
data_nli_answerable_right_test_df = convert_question_and_answer_to_hypothesis(data_nli_answerable_right_test_df)

data_nli_answerable_right_train_df = move_to_column_number(data_nli_answerable_right_train_df, "hypothesis", 3)
data_nli_answerable_right_val_df = move_to_column_number(data_nli_answerable_right_val_df, "hypothesis", 3)
data_nli_answerable_right_test_df = move_to_column_number(data_nli_answerable_right_test_df, "hypothesis", 3)

In [ ]:
data_nli_answerable_wrong_train_df = convert_question_and_answer_to_hypothesis(data_nli_answerable_wrong_train_df)
data_nli_answerable_wrong_val_df = convert_question_and_answer_to_hypothesis(data_nli_answerable_wrong_val_df)
data_nli_answerable_wrong_test_df = convert_question_and_answer_to_hypothesis(data_nli_answerable_wrong_test_df)

data_nli_answerable_wrong_train_df = move_to_column_number(data_nli_answerable_wrong_train_df, "hypothesis", 3)
data_nli_answerable_wrong_val_df = move_to_column_number(data_nli_answerable_wrong_val_df, "hypothesis", 3)
data_nli_answerable_wrong_test_df = move_to_column_number(data_nli_answerable_wrong_test_df, "hypothesis", 3)

In [ ]:
data_nli_unanswerable_right_train_df = convert_question_and_answer_to_hypothesis(data_nli_unanswerable_right_train_df)
data_nli_unanswerable_right_val_df = convert_question_and_answer_to_hypothesis(data_nli_unanswerable_right_val_df)
data_nli_unanswerable_right_test_df = convert_question_and_answer_to_hypothesis(data_nli_unanswerable_right_test_df)

data_nli_unanswerable_right_train_df = move_to_column_number(data_nli_unanswerable_right_train_df, "hypothesis", 3)
data_nli_unanswerable_right_val_df = move_to_column_number(data_nli_unanswerable_right_val_df, "hypothesis", 3)
data_nli_unanswerable_right_test_df = move_to_column_number(data_nli_unanswerable_right_test_df, "hypothesis", 3)

In [ ]:
data_nli_unanswerable_wrong_train_df = convert_question_and_answer_to_hypothesis(data_nli_unanswerable_wrong_train_df)
data_nli_unanswerable_wrong_val_df = convert_question_and_answer_to_hypothesis(data_nli_unanswerable_wrong_val_df)
data_nli_unanswerable_wrong_test_df = convert_question_and_answer_to_hypothesis(data_nli_unanswerable_wrong_test_df)

data_nli_unanswerable_wrong_train_df = move_to_column_number(data_nli_unanswerable_wrong_train_df, "hypothesis", 3)
data_nli_unanswerable_wrong_val_df = move_to_column_number(data_nli_unanswerable_wrong_val_df, "hypothesis", 3)
data_nli_unanswerable_wrong_test_df = move_to_column_number(data_nli_unanswerable_wrong_test_df, "hypothesis", 3)

# Add label: entailment & contradiction

In [ ]:
data_nli_answerable_right_train_df['label'] = 'entailment'
data_nli_answerable_right_val_df['label'] = 'entailment'
data_nli_answerable_right_test_df['label'] = 'entailment'

data_nli_answerable_right_train_df = move_to_column_number(data_nli_answerable_right_train_df, "label", 4)
data_nli_answerable_right_val_df = move_to_column_number(data_nli_answerable_right_val_df, "label", 4)
data_nli_answerable_right_test_df = move_to_column_number(data_nli_answerable_right_test_df, "label", 4)

In [ ]:
data_nli_answerable_wrong_train_df['label'] = 'contradiction'
data_nli_answerable_wrong_val_df['label'] = 'contradiction'
data_nli_answerable_wrong_test_df['label'] = 'contradiction'

data_nli_answerable_wrong_train_df = move_to_column_number(data_nli_answerable_wrong_train_df, "label", 4)
data_nli_answerable_wrong_val_df = move_to_column_number(data_nli_answerable_wrong_val_df, "label", 4)
data_nli_answerable_wrong_test_df = move_to_column_number(data_nli_answerable_wrong_test_df, "label", 4)

In [ ]:
data_nli_unanswerable_right_train_df['label'] = 'entailment'
data_nli_unanswerable_right_val_df['label'] = 'entailment'
data_nli_unanswerable_right_test_df['label'] = 'entailment'

data_nli_unanswerable_right_train_df = move_to_column_number(data_nli_unanswerable_right_train_df, "label", 4)
data_nli_unanswerable_right_val_df = move_to_column_number(data_nli_unanswerable_right_val_df, "label", 4)
data_nli_unanswerable_right_test_df = move_to_column_number(data_nli_unanswerable_right_test_df, "label", 4)

In [ ]:
data_nli_unanswerable_wrong_train_df['label'] = 'contradiction'
data_nli_unanswerable_wrong_val_df['label'] = 'contradiction'
data_nli_unanswerable_wrong_test_df['label'] = 'contradiction'

data_nli_unanswerable_wrong_train_df = move_to_column_number(data_nli_unanswerable_wrong_train_df, "label", 4)
data_nli_unanswerable_wrong_val_df = move_to_column_number(data_nli_unanswerable_wrong_val_df, "label", 4)
data_nli_unanswerable_wrong_test_df = move_to_column_number(data_nli_unanswerable_wrong_test_df, "label", 4)

# Concat the right and wrong NLI to one NLI dataset

In [ ]:
data_nli_train_df_final = pd.concat([data_nli_answerable_right_train_df, 
                                     data_nli_answerable_wrong_train_df,
                                     data_nli_unanswerable_right_train_df,
                                     data_nli_unanswerable_wrong_train_df], axis=0, ignore_index=True)

data_nli_val_df_final = pd.concat([data_nli_answerable_right_val_df, 
                                   data_nli_answerable_wrong_val_df,
                                   data_nli_unanswerable_right_val_df,
                                   data_nli_unanswerable_wrong_val_df], axis=0, ignore_index=True)

data_nli_test_df_final = pd.concat([data_nli_answerable_right_test_df, 
                                    data_nli_answerable_wrong_test_df,
                                    data_nli_unanswerable_right_test_df,
                                    data_nli_unanswerable_wrong_test_df], axis=0, ignore_index=True)

In [ ]:
data_nli_answerable_wrong_test_df

In [ ]:
'''
for i in range(len(data_nli_answerable_right_test_df)):
    print(f"Iterasi: {i}")
    print(f"answer: {data_nli_answerable_right_test_df['answer'][i]}")
    print("NER")
    print(data_nli_answerable_right_test_df['ner_tag_answer'][i])
    print(data_nli_answerable_right_test_df['ner_tag_premise'][i])
    print("Chunking")
    print(data_nli_answerable_right_test_df['chunking_tag_answer'][i])
    print(data_nli_answerable_right_test_df['chunking_tag_premise'][i])
    print()
'''

In [ ]:
print("TRAIN FINAL")
print(len(data_nli_train_df_final))
print()

print("VAL FINAL")
print(len(data_nli_val_df_final))
print()

print("TEST FINAL")
print(len(data_nli_test_df_final))

# Convert to DataFrame format to CSV

In [ ]:
data_nli_train_df_final.to_csv(f"{NAME}_nli_train_df.csv", index=False)
data_nli_val_df_final.to_csv(f"{NAME}_nli_val_df.csv", index=False)
data_nli_test_df_final.to_csv(f"{NAME}_nli_test_df.csv", index=False)

In [ ]:
print("PROGRAM FINISHED")